<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1_threads_Cpp_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **C++**

Solución implementada utilizando un *aproach* funcional, priorizando la inmutabilidad de los datos

In [23]:
%%writefile matrix_mult.cpp
#include <iostream>
#include <vector>
#include <string>
#include <random>
#include <utility>
#include <thread>
#include <stdlib.h>

#define MIN_MATRIX_SIZE 5
#define MAX_MATRIX_SIZE 20
#define MIN_MATRIX_CELL_VALUE -32
#define MAX_MATRIX_CELL_VALUE 32

// Definimos a la matriz como un tipo de datos vector de vector de t_matrix
// t_matrix es el tipo de dato que contiene la matriz
// está definido en minuscula para no conufndir con los tipos de datos genéricos ej <T>
typedef int t_matrix;
typedef std::vector<std::vector<t_matrix>> Matrix;

int validateArgs(int& argc, char* argv[]);
Matrix matrixGenerator(int size, bool populate = true);
Matrix matrixGenerator(int row_size, int col_size, bool populate = true);
Matrix secuentialMult(const Matrix&, const Matrix&);
Matrix concurrentMult(const Matrix&, const Matrix&);
void printMatrix(const std::string alias, const Matrix&);

int main(int argc, char* argv[])
{
    int size = validateArgs(argc, argv);

    Matrix matrix_a  {matrixGenerator(size)};
    Matrix matrix_b  {matrixGenerator(size)};
    Matrix matrix_cs {secuentialMult(matrix_a, matrix_b)};
    Matrix matrix_ch {concurrentMult(matrix_a, matrix_b)};

    printMatrix("A", matrix_a);
    printMatrix("B", matrix_b);
    printMatrix("CS", matrix_cs);
    printMatrix("CH", matrix_ch);
    std::cout << "Las matrices CS y CH son iguales? "
        // El operador == en vectores compara todos los elementos del vector
        << (matrix_cs == matrix_ch ? "Si" : "No")
        << std::endl;

    return EXIT_SUCCESS;
}

// Valida los argumentos y devuelve el tamaño de la matrices a crear
int validateArgs(int& argc, char* argv[])
{
    if (argc != 2)
    {
        std::cout << "Cantidad de argumentos incorrectos" << std::endl;
        exit(EXIT_FAILURE);
    }

    int size = atoi(argv[1]);
    if (size < MIN_MATRIX_SIZE || size > MAX_MATRIX_SIZE)
    {
        std::cout << "El tamaño de la matriz debe estar entre "
            << MIN_MATRIX_SIZE << " y " << MAX_MATRIX_SIZE << std::endl;

        exit(EXIT_FAILURE);
    }

    return size;
}

// Genera una matriz de igual tamaño de filas y columnas
Matrix matrixGenerator(int size, bool populate)
{
    // "Movemos" al objeto en vez de realizar una copia
    return std::move(matrixGenerator(size, size, populate));
}

// Genera una matriz y las llena de datos de ser necesario
Matrix matrixGenerator(int row_size, int col_size, bool populate)
{
    Matrix aux(row_size, std::vector<t_matrix>(col_size));

    if (populate)
    {
        std::random_device rd; // Obtenemos un número random del hardware
        std::mt19937 gen(rd()); // Generamos la semilla (seed)
        std::uniform_int_distribution<> distr(MIN_MATRIX_CELL_VALUE, MAX_MATRIX_CELL_VALUE); // Definimos el rango

        for (int i=0; i < row_size; ++i)
            for (int j=0; j < col_size; ++j)
                aux[i][j] = distr(gen);
    }

    // "Movemos" al objeto en vez de realizar una copia
    return std::move(aux);
}

// Realiza la multiplicación entre 2 matrices y devuelve una nueva de manera secuencial
Matrix secuentialMult(const Matrix& a, const Matrix& b)
{
    Matrix aux = matrixGenerator(a.size(), false);

    for (int i=0; i < a.size(); ++i)
        for (int j=0; j < b[0].size(); ++j)
            for (int k=0; k < a.size(); ++k)
                aux[i][j] += a[i][k] * b[k][j];    

    // "Movemos" al objeto en vez de realizar una copia
    return std::move(aux);
}

// Realiza la multiplicación entre 2 matrices y devuelve una nueva de manera concurrente
Matrix concurrentMult(const Matrix& a, const Matrix& b)
{
    Matrix aux = matrixGenerator(a.size(), false);
    std::vector<std::thread> workers;

    for (int i; i < a.size(); ++i)
    {
        workers.push_back(std::thread(
                // Función anónima o lambda que realiza la asignación y multiplicación
                [&aux, &a, &b, i]()
                {
                    for (int j=0; j < b[0].size(); ++j)
                        for (int k=0; k < a.size(); ++k)
                            aux[i][j] += a[i][k] * b[k][j];
                }
            )
        );
    }
    
    for(auto& worker : workers)
    {
        worker.join();
    }

    // "Movemos" al objeto en vez de realizar una copia
    return std::move(aux);
}

// Imprime la matriz con su alias
void printMatrix(const std::string alias, const Matrix& m)
{
    std::cout << "Matrix " << alias << ":" << std::endl;
    
    for(const auto& row : m)
    {
        for(const auto& col : row)
        {
            std::cout << "\t" << col << "\t";
        }
        std::cout << std::endl;
    }
}


Overwriting matrix_mult.cpp


Compilamos el programa, indicando que utilice el módulo pthread para poder utilizar hilos

In [24]:
!g++ matrix_mult.cpp -pthread -std=c++14 -o matrix_mult.out

## Pruebas de validación de parametros:

In [25]:
!./matrix_mult.out

Cantidad de argumentos incorrectos


In [26]:
!./matrix_mult.out 1

El tamaño de la matriz debe estar entre 5 y 20


In [27]:
!./matrix_mult.out 21

El tamaño de la matriz debe estar entre 5 y 20


## Ejecución exitosa:

In [30]:
!./matrix_mult.out 6

Matrix A:
	-32		-28		-19		-7		28		-9	
	19		-5		-14		-8		-21		-21	
	-18		20		22		-30		-1		-12	
	24		-17		0		14		17		-15	
	7		-7		11		-11		-15		7	
	-23		8		4		3		-29		1	
Matrix B:
	-30		-28		-6		-14		-6		26	
	-8		-29		20		-26		0		-31	
	-2		6		-22		24		18		16	
	-26		13		-3		16		16		14	
	10		-19		-28		8		-8		19	
	16		27		16		29		-16		20	
Matrix CS:
	1540		728		-857		571		-342		-14	
	-840		-743		370		-1377		10		-506	
	914		-639		-50		-576		224		-1415	
	-1018		-725		-1242		31		184		1370	
	72		404		141		255		-12		276	
	266		1053		1029		55		474		-1271	
Matrix CH:
	1540		728		-857		571		-342		-14	
	-840		-743		370		-1377		10		-506	
	914		-639		-50		-576		224		-1415	
	-1018		-725		-1242		31		184		1370	
	72		404		141		255		-12		276	
	266		1053		1029		55		474		-1271	
Las matrices CS y CH son iguales? Si
